# Polygon Bridging Behaviour
> "What do users do when they bridge to Polygon?"

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [polygon]

In [2]:
#hide
#Imports & settings
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline
#%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))

dplyr.summarise.inform,[RTYPES.NILSXP]


In [3]:
#hide
%%R
#Grab base query from Flipside
df_group1 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/ce6a4190-132d-46fc-96a3-f70127512f85/data/latest', simplifyDataFrame = TRUE)
df_group2 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/e391cb58-6f9d-4f90-9fd9-bd07a268997b/data/latest', simplifyDataFrame = TRUE)
df_group3 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/8eca21f3-20ef-4ce9-88e6-ce03591b3a71/data/latest', simplifyDataFrame = TRUE)
df_group4 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/c03db498-3094-45f3-93c0-30193e111b14/data/latest', simplifyDataFrame = TRUE)

#union all the three query groups
df <- df_group1 %>%
  bind_rows(df_group2) %>%
  bind_rows(df_group3) %>%
  bind_rows(df_group4)

rm(list = c("df_group1", "df_group2", "df_group3", "df_group4"))

#Change the date to date format
df$BLOCK_TIMESTAMP <- parse_datetime(df$BLOCK_TIMESTAMP)

#lower case the column names
names(df)<-tolower(names(df))

#grab labels
labels <- read_csv("https://raw.githubusercontent.com/scottincrypto/analytics/master/data/bridge_labels.csv")

#join labels
df <- df %>%
  left_join(labels)

#replace nas with Unknown
df$Label <- replace_na(df$Label, "Unknown")
df$Class <- replace_na(df$Class, "Unknown")

#create the RHS of the sankey link table
rhs_link <- df %>% group_by(symbol, Label) %>%
  summarise(total = sum(nxt_amount_usd)) %>%
  rename(source = symbol, target = Label)

#create the LHS of the sankey link table
lhs_link <- df %>% group_by(symbol) %>%
  summarise(total = sum(amount_usd)) %>%
  mutate(source = "Bridge In") %>%
  select(source, symbol, total) %>%
  rename(target = symbol)

#Join the table together
link_table <- lhs_link %>% bind_rows(rhs_link) 

#Create a list of the nodes
nodes <- link_table %>% rename(node = source) %>%
  select(node) %>% 
  bind_rows(link_table %>% rename(node = target) %>% select(node)) %>% 
  distinct(node) %>%
  mutate(index = row_number()-1)

#join the index back into the link table
link_table <- link_table %>%
  left_join(nodes, by=c("source" = "node")) %>%
  rename(source_index = index)
link_table <- link_table %>%
  left_join(nodes, by=c("target" = "node")) %>%
  rename(target_index = index)

#Grab the top Coins chart
coin_chart = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/55d223eb-f8f4-4e0a-97bb-6a0eb5a22b7d/data/latest', simplifyDataFrame = TRUE)

coin_chart <- coin_chart %>% 
  mutate(total = sum(AMOUNT_USD),
         Percentage = round(AMOUNT_USD / total * 100,0)) %>%
  rename('Bridged Token' = SYMBOL,'USD Amount' = AMOUNT_USD) %>%
  select('Bridged Token', 'USD Amount', Percentage) %>%
  arrange(desc('USD Amount'))


Rows: 77 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): nxt_to_address, Label, Class

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Joining, by = "nxt_to_address"


# Introduction

This post seeks to answer the following questions:    
  
*Where do people go when they bridge to Polygon from Ethereum? What are the 10 most popular first destinations for Polygon addresses that have just bridged from Ethereum? What has this been for each day in the past month?*
  
Polygon operates as a sidechain to Ethereum, with funds moved to & from Polygon via a series of bridges.  These bridges accept tokens on the Ethereum side, then create wrapped versions of the same tokens on the Polygon side. Users are attracted to Polygon by the low fees & fast transactions speeds relative to Ethereum.  In the absence of fiat onramps, bridges are the only way into the ecosystem.  There are a number of bridges in operation by different operators, but this analysis will focus on the official Polygon bridge provided by the protocol.  



## About the Data

This dataset looks at all of the transactions coming into Polygon via the Polygon Bridge over the last month.  It then looks at the *next* transaction each wallet undertook in the currency that was bridged in, and classifies the destination of that transaction.  In examining the data, a couple of observations were noted:
- Often the first transaction was a very small swap for MATIC - the token used for gas on Polygon
- Most of the coins bridged were either USDC, USDT, WBTC or WETH.  Only 3% were other coins, and there was a very long list of these other coins.

To simplify the analysis, we excluded any next transaction with a value of less than USD 10, and looked at the transaction *after* this instead.  This lets us see the intent of the user more than looking at their initial MATIC swap transaction instead.  We also limited our analysis to the 4 major coins above - this accounts for 97% of the funds bridged (as per the table below) and allowed us to simplify the charts for better insights.

All data was sourced from [Flipside Crypto](https://flipsidecrypto.com)


In [4]:
#hide_input
%R coin_chart

,Bridged Token,USD Amount,Percentage
1,USDC,4.457159e+08,45.0
2,USDT,2.783901e+08,28.0
3,WBTC,1.622389e+08,16.0
4,WETH,8.141258e+07,8.0
5,other,2.897250e+07,3.0


# Top Destinations

The destination addresses & contracts were classified by project or protocol to understand where users were going after bridging to Polygon.  We managed to classify over 70 of these destination contracts to account for over 75% of the value moved.  There was a very long tail (over 23k addresses) accounting for the remaining 25% of the value - these are labelled as "Unknown" in the data going forward.

The graph below shows the next destination of the bridged funds.  Aave is the top destination, with a lot of value coming into Polygon to take part in borrowing & lending activities.  A number of familiar names from the Ethereum ecosystem are in the list - Curve, 1inch, Balance, Sushi - but there are a few non-Ethereum specific names in the list - Quickswap, Iron Finance, Polynetwork.  An interesting find in this list are the bridges - a signficant portion of funds were immediately bridged out of Polygon after being bridged in.  This was done via the Polygon Bridge ("Bridge Out") or via the Allbridge facility.  It's possible users are using Allbridge to get to chains such as Solana or Binance using Polygon as an intermediate step.

In [24]:
#hide_input
# Plot the top 10
df_p = %R df %>% group_by(Label) %>%  summarise(total = sum(nxt_amount_usd)) %>%  arrange(desc(total)) 
fig = px.bar(df_p
             , x = "Label"
             , y = "total"
             , labels=dict(Label="Destination", total="USD Amount")
             , title= "Next Destination of Bridged Funds on Polygon"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Amount (USD)')
fig.update_xaxes(title_text='Destination')
fig.show()

# Flow of Funds
## By Destination

The graph below shows the flow of funds from bridge, by token bridged, into the destination protocols shown above.  We can see the large flows into Aave - interestingly most of the Aave flow is USDC, USDT or WBTC.  WETH makes up only a small portion of the flow.  WETH yields are usually low on Aave, but WBTC yields are too.  Low yields do not show the full picture of user behaviour here.

In [27]:
#hide_input
# Sankey by Label
label = %R nodes %>% select(node)
source = %R link_table %>% select(source_index)
target = %R link_table %>% select(target_index)
value = %R link_table%>% select(total)

label = label['node'].to_list()
source = source['source_index'].to_list()
target = target['target_index'].to_list()
value = value['total'].to_list()

# data to dict, dict to sankey
link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=50, thickness=5)
data = go.Sankey(link = link, node=node)
fig = go.Figure(data)
fig.update_layout(width=800, height=800/1.618, template="simple_white", title="Flow of Funds from Bridge to Destination (amounts in USD)")
fig.show()

## By Use Case

To simplify the above graph, each destination was classified into a broad use case grouping.  This is shown in the graph below.  Users coming to Polygon for Defi activities (Aave, Iron Finance etc) account for the largest grouping.  Here we see the amount of funds bridge immediately back out of Polygon - 11% of funds bridge in are immediately bridged out.  Another observation from this graph is that there is a small amount of hodling occuring - this is seen where the inflows to each token are less than the outflows. 

In [7]:
#hide
%%R

#Do the same graph by Class

#create the RHS of the sankey link table
rhs_link <- df %>% group_by(symbol, Class) %>%
  summarise(total = sum(nxt_amount_usd)) %>%
  rename(source = symbol, target = Class)

#create the LHS of the sankey link table
lhs_link <- df %>% group_by(symbol) %>%
  summarise(total = sum(amount_usd)) %>%
  mutate(source = "Bridge In") %>%
  select(source, symbol, total) %>%
  rename(target = symbol)

#Join the table together
link_table <- lhs_link %>% bind_rows(rhs_link) 

#Create a list of the nodes
nodes <- link_table %>% rename(node = source) %>%
  select(node) %>% 
  bind_rows(link_table %>% rename(node = target) %>% select(node)) %>% 
  distinct(node) %>%
  mutate(index = row_number()-1)

#join the index back into the link table
link_table <- link_table %>%
  left_join(nodes, by=c("source" = "node")) %>%
  rename(source_index = index)
link_table <- link_table %>%
  left_join(nodes, by=c("target" = "node")) %>%
  rename(target_index = index)

In [26]:
#hide_input

# Sankey by Class
label = %R nodes %>% select(node)
source = %R link_table %>% select(source_index)
target = %R link_table %>% select(target_index)
value = %R link_table%>% select(total)

label = label['node'].to_list()
source = source['source_index'].to_list()
target = target['target_index'].to_list()
value = value['total'].to_list()

# data to dict, dict to sankey
link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=50, thickness=5)
data = go.Sankey(link = link, node=node)
fig = go.Figure(data)
fig.update_layout(width=800, height=800/1.618, template="simple_white", title="Flow of Funds from Bridge to Use Case (amounts in USD)")
fig.show()

# Most Popular Destinations by Day

The chart below shows, by day, the ranking of each of the destinations by the USD amount sent to them.  This chart is very busy and a littled difficult to intepret.  To target a particular protocol, double-click on the protocol in the legend and the graph will isolate just that protocol.  Clicking on another protocol after this will add it to the graph.  Double-click again to return to all values.

Aave is consistently the top performer, being always ranked at number 4 or higher.  The unknown category also features highly - indicating that there are a wide variety of destinations on Polygon and it's not just a small handful capturing all the action.  The behaviour with bridges continues to yield interesting insight - Allbridge has risen to be a consistent top 5 destination in the last 2 weeks - perhaps this is coincident with the rise in interest in Solana, as Allbridge can be used to bridge to this chain.

In [14]:
#hide
%%R
#top 10 rank by day
top_10_table <- df %>% mutate(date = floor_date(block_timestamp, unit = "days")) %>%
  group_by(date, Label) %>%
  summarise(dest_total = sum(nxt_amount_usd)) %>%
  ungroup() %>%
  group_by(date) %>%
  mutate(rank = rank(desc(dest_total))) %>%
  ungroup() %>%
#  filter(rank <= 10) %>%
  arrange(date, rank)

In [18]:
#hide_input

#aave top graph - need rank by day first
#Top 5 pools by week for 6 months
df_p = %R top_10_table %>% arrange(date, rank)

fig = px.line(df_p, x="date", y="rank", color='Label',
              template="simple_white", width=1200, height=1200/1.618, 
              title= 'Most Popular Desination, Ranked, by Day',
              labels=dict(date="Date", rank="Rank", Label="Destination"))
fig.update_yaxes(autorange="reversed")
fig.update_traces(mode="lines+markers")
fig.update_yaxes(tick0=1, dtick=1)
#fig.update_layout(legend=dict(
#    yanchor="bottom",
#    y=0.01,
#    xanchor="left",
#    x=0.01
#))
fig.show()

# Most Popular Use Cases by Day

To simplify the above graph, we used the broader classes of use cases rather than the destination protcols.  Here we see that Defi usage is on average the highest - this contains Aave.  Users are obviously keen to generate yield from their funds in Polygon, taking advantage of the low fees relative to Ethereum to deposit their funds.  We also see that funds jumping straight to dex swaps has dropped off - 3-4 weeks ago this was the highest use case, now it is amongst the lowest.

In [21]:
#hide
%%R
#top 10 rank by day
top_10_table <- df %>% mutate(date = floor_date(block_timestamp, unit = "days")) %>%
  group_by(date, Class) %>%
  summarise(dest_total = sum(nxt_amount_usd)) %>%
  ungroup() %>%
  group_by(date) %>%
  mutate(rank = rank(desc(dest_total))) %>%
  ungroup() %>%
  filter(rank <= 10) %>%
  arrange(date, rank)

In [23]:
#hide_input

df_p = %R top_10_table %>% arrange(date, rank)

fig = px.line(df_p, x="date", y="rank", color='Class',
              template="simple_white", width=1200, height=1200/1.618, 
              title= 'Most Popular Use Case, Ranked, by Day',
              labels=dict(rank="Rank", date="Date", Class="Use Case"))
fig.update_yaxes(autorange="reversed")
fig.update_traces(mode="lines+markers")
fig.update_yaxes(tick0=1, dtick=1)
#fig.update_layout(legend=dict(
#    yanchor="bottom",
#    y=0.01,
#    xanchor="left",
#    x=0.01
#))
fig.show()

# Conclusions

The Polygon Bridge allows users to bring funds from Ethereum into the Polygon ecosystem.  We have looked at what users do once they bridge funds in.  We have discovered the following insights:
- 97% of value bridged in is 4 tokens - USDT, USDC, WBTC, WETH - with a long tail of other tokens.
- The top single destination for bridged funds is depositing on Aave.
- There is a mix of Ethereum-based protocols used (Aave, Curve, Balancer, Sushi) and protocols that are based in other chains like Polygon & BSC (Quickswap, Iron Finance).  The former indicates users bridging to Polygon for the lower fee environment, but the latter indicates there may be pull due to some unique opportunities on the ecosystem.
- We saw 11% of funds bridged in immediately bridged out in the next transaction.  It's possible that Polygon is being used as an intermediate step to bridge to other ecosystems.
- We looked at the most popular destinations and use cases by day - Defi usage (depositing, yield seeking) is consistently the most common use case, driven by Aave.